In [1]:
import sys; print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [89]:
import pandas as pd 
import numpy as np
from datetime import datetime , timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [53]:
from lightgbm import LGBMRegressor

In [54]:
# Importer un CSV
df = pd.read_csv("D:/GitHub/Dataset/g-research-crypto-forecasting/train.csv")


In [55]:
ad = pd.read_csv("D:/GitHub/Dataset/g-research-crypto-forecasting/asset_details.csv")

In [56]:
#train_2020 = train.loc[(train['timestamp'] >= 1577873958) & (train['timestamp'] < 1609455600)]

### Data Exploration

In [57]:
df.shape

(24236806, 10)

In [101]:
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,datetime
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218,2018-01-01 01:01:00
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,2018-01-01 01:01:00
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643,2018-01-01 01:01:00
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922,2018-01-01 01:01:00
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264,2018-01-01 01:01:00
...,...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN,2021-09-21 02:00:00
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN,2021-09-21 02:00:00
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN,2021-09-21 02:00:00
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN,2021-09-21 02:00:00


In [59]:
ad

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [60]:
df.dtypes

timestamp      int64
Asset_ID       int64
Count        float64
Open         float64
High         float64
Low          float64
Close        float64
Volume       float64
VWAP         float64
Target       float64
dtype: object

In [61]:
first_step = 0
step = 1000000
df = pd.DataFrame()

for rows_interval in range(first_step, train.shape[0], step):
    tmp_df = train.iloc[rows_interval:rows_interval + step]
    tmp_df['datetime'] = tmp_df.timestamp.apply(datetime.fromtimestamp)
    df = pd.concat([df, tmp_df], axis=0)

In [62]:
df.isnull().sum(axis=0)

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
datetime          0
dtype: int64

In [224]:
df_n = df.loc[df['Target'].isnull()]

In [225]:
(df_n['Asset_ID'].value_counts() / df['Asset_ID'].value_counts() ) *100

0      0.690563
1      0.015540
2      0.248831
3      1.045334
4     12.531616
5      0.117741
6      0.017381
7      0.477980
8     12.129233
9      0.026636
10     5.969453
11    14.083847
12     3.475251
13     1.121010
Name: Asset_ID, dtype: float64

In [208]:
#df_n['Month'] = pd.DatetimeIndex(df_n['datetime']).month
#df_n['Year'] = pd.DatetimeIndex(df_n['datetime']).year

In [209]:
#df_n

In [210]:
#df_n['Month'].value_counts()

In [211]:
#df_n['Year'].value_counts()

In [252]:
df_n['datetime-1']  = df_n['datetime'] - timedelta(minutes=1)
df_n['datetime+1']  = df_n['datetime'] + timedelta(minutes=1)

In [253]:
df_n = df.loc[df['Target'].isnull()]

df_n['datetime-1']  = df_n['datetime'] - timedelta(minutes=1)
df_n['datetime+1']  = df_n['datetime'] + timedelta(minutes=1)

dfn_2 =  pd.DataFrame()
dfn_2 = pd.merge(df_n, df[['Asset_ID','datetime','Target']], how = 'left', left_on = ['Asset_ID','datetime-1'] , right_on = ['Asset_ID','datetime'])
dfn_2 = dfn_2.rename(columns = {'Target_y' : 'Target_t-1' , 'Target_x' : 'Target' , 'datetime_x' : 'datetime'})
dfn_2 = dfn_2.drop(columns = {'datetime_y', 'datetime-1'})

dfn_3 =  pd.DataFrame()
dfn_3 = pd.merge(dfn_2, df[['Asset_ID','datetime','Target']], how = 'left', left_on = ['Asset_ID','datetime+1'] , right_on = ['Asset_ID','datetime'])
dfn_3 = dfn_3.rename(columns = {'Target_y' : 'Target t+1' , 'Target_x' : 'Target' , 'datetime_x' : 'datetime'})
dfn_3 = dfn_3.drop(columns = {'datetime_y', 'datetime+1'})


In [254]:
dfn_3['if_target_null'] = np.where(dfn_3['Target'].isnull(), 1,0)
dfn_3['if_target_null_-1'] = np.where(dfn_3['Target_t-1'].isnull(), 1,0)
dfn_3['if_target_null_+1'] = np.where(dfn_3['Target t+1'].isnull(), 1,0)

In [255]:
df_n_analysis = dfn_3.groupby(['Asset_ID'])['if_target_null','if_target_null_-1','if_target_null_+1'].sum()

df_n_analysis['%_of_null_before_a_null'] = (df_n_analysis['if_target_null_-1'] / df_n_analysis['if_target_null']) * 100
df_n_analysis['%_of_null_after_a_null'] = (df_n_analysis['if_target_null_+1'] / df_n_analysis['if_target_null']) * 100

In [256]:
df_n_analysis

,if_target_null,if_target_null_-1,if_target_null_+1,%_of_null_before_a_null,%_of_null_after_a_null
Asset_ID,,,,,
0,13451,2379,7980,17.686417,59.326444
1,304,228,271,75.000000,89.144737
2,4861,809,2877,16.642666,59.185353
3,18735,3906,11498,20.848679,61.371764
4,145014,64761,107137,44.658447,73.880453
5,2302,485,1395,21.068636,60.599479
6,340,233,286,68.529412,84.117647
7,9326,1268,5327,13.596397,57.119880
8,193198,94051,148271,48.681146,76.745618
